In [32]:
import gurobipy as gp
from gurobipy import GRB

### 决策变量
- 产品I：由A1、A2完成A工序的产品x1、x2件，由B1、B2、B3完成B工序的产品x3、x4、x5件
- 产品II：由A1、A2完成A工序的产品x6、x7件，由B1完成B工序的产品x8件
- 产品III：由A2、B2完成工序的产品都为x9件
### 目标函数：
$$\sum_{i=1}^{7}(单价_i-原料费_i)\times x_i - 设备费用$$
### 约束：
- 决策变量定义
$$ x_1+x_2 = x_3+x_4+x_5 $$
$$ x_6+x_7 = x_8 $$
- 设备有效台时

In [33]:
m = gp.Model("设备工时整数优化")

In [34]:
x = m.addVars(9, name="x", vtype=GRB.INTEGER)

In [35]:
m.addConstr(x[0] + x[1] == x[2] + x[3] + x[4])
m.addConstr(x[5] + x[6] == x[7])

<gurobi.Constr *Awaiting Model Update*>

In [36]:
profit = [1.25 - 0.25, 2 - 0.35, 2.8 - 0.5]
times = [6000, 10000, 4000, 7000, 4000]
costs = [300, 321, 250, 783, 200]
cost_per_time = [j / i for i, j in zip(times, costs)]

In [37]:
m.setObjective(
    profit[0] * (x[0] + x[1])
    + profit[1] * x[7]
    + profit[2] * x[8]
    - cost_per_time[0] * (x[0] + x[5])
    - cost_per_time[1] * (x[1] + x[6] + x[8])
    - cost_per_time[2] * (x[2] + x[7])
    - cost_per_time[3] * (x[3] + x[8])
    - cost_per_time[4] * x[7],
    sense=GRB.MAXIMIZE,
)

In [38]:
m.addConstr(5 * x[0] + 10 * x[5] <= times[0])
m.addConstr(7 * x[1] + 9 * x[6] + 12 * x[8] <= times[1])
m.addConstr(6 * x[2] + 8 * x[7] <= times[2])
m.addConstr(4 * x[3] + 11 * x[8] <= times[3])
m.addConstr(7 * x[4] <= times[4])

<gurobi.Constr *Awaiting Model Update*>

In [39]:
m.write("IP.lp")

In [40]:
m.optimize()

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[arm] - Darwin 23.4.0 23E224)

CPU model: Apple M1 Pro
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 7 rows, 9 columns and 18 nonzeros
Model fingerprint: 0xa25d6557
Variable types: 0 continuous, 9 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [3e-02, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+03, 1e+04]
Found heuristic solution: objective 1371.2432571
Presolve removed 2 rows and 1 columns
Presolve time: 0.00s
Presolved: 5 rows, 8 columns, 15 nonzeros
Variable types: 0 continuous, 8 integer (0 binary)
Found heuristic solution: objective 2115.9790571

Root relaxation: objective 2.717844e+03, 6 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 2717.84424    

In [41]:
print(f"Objective value: {m.ObjVal}")
print("Optimal solution:")
for i in m.getVars():
    print(f"{i.VarName} = {i.X}", end="\n")

Objective value: 2717.7896
Optimal solution:
x[0] = 1200.0
x[1] = 230.0
x[2] = -0.0
x[3] = 859.0
x[4] = 571.0
x[5] = -0.0
x[6] = 500.0
x[7] = 500.0
x[8] = 324.0
